# COGS 108 - Data Checkpoint

# Names

- Nadia Corral
- Jose Deleon
- Christina Tyagi

<a id='research_question'></a>
# Research Question

*How did changes in the Air Quality Index from 2014 to 2018 effect the amount of respiratory diseases in the Central Valley?*

# Dataset(s)

- Dataset Name: Daily AQI
- Link to the dataset: https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI
- Number of observations: 34675

Each data set provides the daily AQI measurement for every county in CA over the  duration of one year. We are going to combine the datasets in order to get the daily AQI measurements from 2014 to 2018 for cities in the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba. 

- Dataset Name: Primary Care Shortage Areas
- Link to the dataset: https://data.ca.gov/dataset/primary-care-shortage-areas-in-california1/resource/82c9007f-1db4-4e80-83ac-8a40232363e2 
- Number of observations: 542

The data set provides information about the population in regards to poverty rate and the number of physicians in each city to determine if the population lacks access to primary healthcare. They then consider the population-to-physician ratio and population in poverty in order to assign a provider ratio score, a poverty score, and a total score. The total score is used to determine which cities are Primary Care Shortage Areas (PCSAs). We are going to collect the determinations of total scores and PCSAs for cities and average them by county for the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba.

- Dataset Name: Incidence Rate Report for California by County; Lung and Bronchus (All Stages^), 2014-2018
- Link to the dataset: https://gis.cdc.gov/Cancer/USCS/#/StateCounty/
- Number of observations: 19

The data set provides the rate of new lung and bronchus cancers from 2014-2018 for every county in CA. We are going to collect the rate of new lung and and bronchus cancers for the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba.

- Dataset Name: 2014-2020 Final Deaths by Year by County
- Link to the dataset: https://data.chhs.ca.gov/dataset/death-profiles-by-county/resource/579cc04a-52d6-4c4c-b2df-ad901c9049b7
- Number of observations: 1425

The data set provides the amount of deaths per year due to respiratory disease for every county in CA. We are going to collect the amount of deaths caused by respiratory disease from 2014 to 2018 for the following counties: Butte, Colusa, Glenn, Fresno, Kern, Kings, Madera, Merced, Placer, San Joaquin, Sacramento, Shasta, Solano, Stanislaus, Sutter, Tehama, Tulare, Yolo and Yuba.


# Setup

In [ ]:
import pandas as pd
import numpy as np

# Data Cleaning: Primary Care Shortage Areas

In [ ]:
#load primary care shortage areas csv
pcsa = pd.read_csv('https://data.chhs.ca.gov/dataset/061494a3-e8c7-4615-a22f-b2851d44eb09/resource/0ba7c904-2302-400a-ba27-b8e8e5c1ab4a/download/pcsa.csv')
pcsa.head()

In [ ]:
#filter out the counties to only get counties in the central valley
pcsa_sub = pcsa.loc[(pcsa['MSSA_COUNTY'] == 'Butte') | (pcsa['MSSA_COUNTY'] == 'Colusa') | (pcsa['MSSA_COUNTY'] == 'Glenn') | 
(pcsa['MSSA_COUNTY'] == 'Fresno') | (pcsa['MSSA_COUNTY'] == 'Kern') | 
(pcsa['MSSA_COUNTY'] == 'Kings') | (pcsa['MSSA_COUNTY'] == 'Madera') | 
(pcsa['MSSA_COUNTY'] == 'Merced') | (pcsa['MSSA_COUNTY'] == 'Placer') | 
(pcsa['MSSA_COUNTY'] == 'San Joaquin') | (pcsa['MSSA_COUNTY'] == 'Sacramento') | 
(pcsa['MSSA_COUNTY'] == 'Shasta') | (pcsa['MSSA_COUNTY'] == 'Solano') | 
(pcsa['MSSA_COUNTY'] == 'Stanislaus') | (pcsa['MSSA_COUNTY'] == 'Sutter') | 
(pcsa['MSSA_COUNTY'] == 'Tehama') | (pcsa['MSSA_COUNTY'] == 'Tulare') | 
(pcsa['MSSA_COUNTY'] == 'Yolo') | (pcsa['MSSA_COUNTY'] == 'Yuba')
]
pcsa_sub

In [ ]:
# only keep columns for county, total score and PCSA
pcsa_sub = pcsa_sub[['MSSA_COUNTY', 'Score_Total', 'PCSA']]

# rename county column
pcsa_sub.columns = ['County', 'Score_Total', 'PCSA']

# replace Yes and No PCSA values with binary values
def replace_with_number(value):
   
    value = value.replace("No", 0)
    value = value.replace("Yes", 1)
    
    return value
pcsa_sub = pcsa_sub.apply(replace_with_number)

# see reduced dataset
pcsa_sub
pcsa_sub.shape

In [ ]:
# get average score total and number of PCSA cities/areas within the county 
list_counties = pcsa_sub.County.unique()
pcsa_county = pd.DataFrame(
                   columns=['County', 'Score_Total', 'PCSAs_Total'])
for county in list_counties:
    # add total scores for each county
    ind_score_total = pcsa_sub.loc[(pcsa_sub['County'] == county)]['Score_Total'].sum()
    # add number of PCSA cites/areas
    pcsa_total = pcsa_sub.loc[(pcsa_sub['County'] == county)]['PCSA'].sum()

    pcsa_county = pcsa_county.append({'County': county, 'Score_Total': ind_score_total, 'PCSAs_Total': pcsa_total}, ignore_index=True)

In [ ]:
# final clean data set
pcsa_county

# Data Cleaning: Incidence Rate Report for California by County; Lung and Bronchus (All Stages^), 2014-2018

In [ ]:
#load lung cancer csv
cancer = pd.read_csv('https://raw.githubusercontent.com/cgtyagi/Group062data/main/LungCancer-Sheet%201-Table%201-1.csv')
cancer.head()

In [ ]:
#rename the inputs in the area column to just get the county name 
def standardize_county(str_county):
    try: 
        str_county = str_county.strip()
        
        if 'California' in str_county:
            str_county = str_county.replace('County, California', '')
            output = str_county 
        else: 
            output = np.nan
    except:
        output = np.nan

    return output

In [ ]:
cancer['Area'] = cancer['Area'].apply(standardize_county)

In [ ]:
cancer['Area'].unique

In [ ]:
#filter out the counties to only get counties in the central valley
cancer_sub = cancer.loc[(cancer['Area'] == 'Butte ') | (cancer['Area'] == 'Colusa ') | (cancer['Area'] == 'Glenn ') | 
(cancer['Area'] == 'Fresno ') | (cancer['Area'] == 'Kern ') | 
(cancer['Area'] == 'Kings ') | (cancer['Area'] == 'Madera ') | 
(cancer['Area'] == 'Merced ') | (cancer['Area'] == 'Placer ') | 
(cancer['Area'] == 'San Joaquin ') | (cancer['Area'] == 'Sacramento ') | 
(cancer['Area'] == 'Shasta ') | (cancer['Area'] == 'Solano ') | 
(cancer['Area'] == 'Stanislaus ') | (cancer['Area'] == 'Sutter ') | 
(cancer['Area'] == 'Tehama ') | (cancer['Area'] == 'Tulare ') | 
(cancer['Area'] == 'Yolo ') | (cancer['Area'] == 'Yuba ')
]
cancer_sub

In [ ]:
#load deaths csv
deaths = pd.read_csv('https://raw.githubusercontent.com/cgtyagi/Group062data/main/2021-11-29_deaths_final_2014_2020_county_year_sup.csv')
deaths.head()

In [ ]:
#get understanding of shape
deaths.shape

In [ ]:
#filter from years 2014-2018
deaths_sub = deaths.loc[(deaths['Year'] == 2014) | (deaths['Year'] == 2015) | (deaths['Year'] == 2016) | (deaths['Year'] == 2017) | (deaths['Year'] == 2018)]
deaths_sub.head()

# Data Cleaning: 2014-2020 Final Deaths by Year by County

In [ ]:
#checknewshape
deaths_sub.shape

In [ ]:
#filter by cause of death (chronic lower respiratory diseases)
deaths_by_respiratory = deaths_sub.loc[(deaths_sub['Cause'] == 'CLD')]
deaths_by_respiratory.head()

In [ ]:
#filter by relevant counties
deaths_by_respiratory_per_county = deaths_by_respiratory[(deaths_by_respiratory['County'] == 'Butte') | (deaths_by_respiratory['County'] == 'Colusa') | (deaths_by_respiratory['County'] == 'Glenn') | 
(deaths_by_respiratory['County'] == 'Fresno') | (deaths_by_respiratory['County'] == 'Kern') | 
(deaths_by_respiratory['County'] == 'Kings') | (deaths_by_respiratory['County'] == 'Madera') | 
(deaths_by_respiratory['County'] == 'Merced') | (deaths_by_respiratory['County'] == 'Placer') | 
(deaths_by_respiratory['County'] == 'San Joaquin') | (deaths_by_respiratory['County'] == 'Sacramento') | 
(deaths_by_respiratory['County'] == 'Shasta') | (deaths_by_respiratory['County'] == 'Solano') | 
(deaths_by_respiratory['County'] == 'Stanislaus') | (deaths_by_respiratory['County'] == 'Sutter') | 
(deaths_by_respiratory['County'] == 'Tehama') | (deaths_by_respiratory['County'] == 'Tulare') | 
(deaths_by_respiratory['County'] == 'Yolo') | (deaths_by_respiratory['County'] == 'Yuba')]

deaths_by_respiratory_per_county.head()

In [ ]:
#take NaNs in count out of filtered data
deaths_by_respiratory_per_county_clean = deaths_by_respiratory_per_county.dropna(subset = ["Count"]) 
deaths_by_respiratory_per_county_clean.head()

# Data Cleaning: AQI Data for 2014-2018

In [ ]:
"""
Unfortunately this step took a little longer than expected because the individual datasets were so large that the jupyter notebooks 
were not able to properly load them and that was just for one year. This led to us having to manually clean the data a little bit 
within excel. Thankfully once we filtered down to California and the counties within the Central Valley this data became much easier 
to work with.
"""
aqi_2014 = pd.read_csv("AQI data/aqi_2014_final.csv")
aqi_2015 = pd.read_csv("AQI data/aqi_2015_final.csv")
aqi_2016 = pd.read_csv("AQI data/aqi_2016_final.csv")
aqi_2017 = pd.read_csv("AQI data/aqi_2017_final.csv")
aqi_2018 = pd.read_csv("AQI data/aqi_2018_final.csv")
aqiTotal = pd.concat([aqi_2014, aqi_2015, aqi_2016, aqi_2017, aqi_2018], axis=0)
aqiTotal = aqiTotal.rename(columns = {"county Name":"County", "State Name":"State"})
aqiTotal.head()

In [ ]:
#Filters down to only the counties we are interested in 
aqiTotal = aqiTotal[(aqiTotal['County'] == 'Butte') | (aqiTotal['County'] == 'Colusa') | (aqiTotal['County'] == 'Glenn') | 
(aqiTotal['County'] == 'Fresno') | (aqiTotal['County'] == 'Kern') | 
(aqiTotal['County'] == 'Kings') | (aqiTotal['County'] == 'Madera') | 
(aqiTotal['County'] == 'Merced') | (aqiTotal['County'] == 'Placer') | 
(aqiTotal['County'] == 'San Joaquin') | (aqiTotal['County'] == 'Sacramento') | 
(aqiTotal['County'] == 'Shasta') | (aqiTotal['County'] == 'Solano') | 
(aqiTotal['County'] == 'Stanislaus') | (aqiTotal['County'] == 'Sutter') | 
(aqiTotal['County'] == 'Tehama') | (aqiTotal['County'] == 'Tulare') | 
(aqiTotal['County'] == 'Yolo') | (aqiTotal['County'] == 'Yuba')]

In [ ]:
#Gets rid of any potential Nan values
aqiTotal = aqiTotal.dropna(subset = ["AQI"]) 
aqiTotal.head()

In [ ]:
#We sort by county and date
aqiTotal = aqiTotal[["State", "County", "Date", "AQI", "Category", "Defining Parameter"]]
aqiTotal['Date'] = pd.to_datetime(aqiTotal['Date'])
aqiTotal = aqiTotal.sort_values(by=['County', "Date"])
aqiTotal.head()
